In [9]:
import pandas
%reset -f

import sys
sys.path.append("../data/exadata/parquet_dataset/query_tool")

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from query_tool import M100DataClient
from matplotlib import pyplot as plt
import datetime
dataset_path = "../data/m100"
client = M100DataClient(dataset_path)

df = client.query(['ambient', "Supply_Air_Temperature"],
                  columns=['timestamp','node','value'],
                  year_month=["22-02"]) 
df.head()

Retrieving data of type: float


,timestamp,node,value
0,2022-02-07 18:56:20+00:00,44,16.6
1,2022-02-07 18:56:40+00:00,44,16.6
2,2022-02-07 18:57:00+00:00,44,16.4
3,2022-02-07 18:57:20+00:00,44,16.4
4,2022-02-07 18:57:40+00:00,44,16.4


In [10]:
vertiv_df = client.query(["Supply_Air_Temperature"],
                  columns=['timestamp','value'],
                  year_month=["22-02"])
vertiv_df = vertiv_df.rename(columns = {"value": "air_temp" })

ipmi_df = client.query(["ambient"],
                  columns=['timestamp', "node", 'value'],
                  year_month=["22-02"])
ipmi_df = ipmi_df.rename(columns = {"value": "ambient_temp" }).iloc[:100000]


Retrieving data of type: float
Retrieving data of type: float


In [15]:

new_df = vertiv_df.merge(ipmi_df, on = "timestamp")
new_df.head(100)

# new_df["timestamp"] = new_df["timestamp"].apply(lambda ts: ts.as_unit("ns"))


,timestamp,air_temp,node,ambient_temp
0,2022-02-06 23:00:00+00:00,23.5,46,20.799999
1,2022-02-06 23:00:00+00:00,23.5,48,22.600000
2,2022-02-06 23:00:00+00:00,23.5,49,23.000000
3,2022-02-06 23:00:00+00:00,23.5,50,23.000000
4,2022-02-06 23:00:00+00:00,23.5,76,23.600000
...,...,...,...,...
95,2022-02-06 23:00:00+00:00,24.1,82,21.000000
96,2022-02-06 23:00:00+00:00,24.1,519,29.200001
97,2022-02-06 23:00:00+00:00,24.1,521,21.600000
98,2022-02-06 23:00:00+00:00,24.1,523,22.200001


0 days 00:00:00


In [14]:
model = LinearRegression().fit(vertiv_df, ipmi_df)
print(model)

TypeError: float() argument must be a string or a number, not 'Timestamp'